In [1]:
#this notebook is designed to access the unavco gps/gnss ftp system, and from the
#file list generated by their DAIv2 interface it will download all of those files,
#since their download manager would not function properly on any of my browsers.

#In addition to this, this notebook separates the measurements in the file by measurement type
#and maintains the RINEX text format
#there is also a diff test of origional files with the specified index to my created files
#set the flag speceifications

#eventually, I plan to thread this as it takes some time for each yearly list
#at the moment, must be placed in directory above 'Desktop'

In [2]:
import os

In [3]:
import pandas as pd

In [4]:
import numpy as np

In [5]:
from ftplib import FTP

In [6]:
import shutil

In [7]:
import time

In [8]:
#selection/specification by user
dirpath="Desktop/testdir"#setup the directory you would like files to be put, preferrably 1 level away from 
                         #where you want the zip to end up
savepath='qsec'#path for processed files to be zipped
select='pressure'#column to parse out
headerrem="PR"#header to keep

#required files
paths=pd.read_fwf('fileList.jsp.txt',header=None,skiprows=2)#get paths from filelist, from DAIv2


#constants
my_col_lab=['year','month','day','hour','minute','second','pressure','temp dry','relative humidity','wind speed','wind directioin','rain inc','hail inc']
run_diff_test=True


In [9]:
#set up directories
if not os.path.exists(dirpath):
    os.makedirs(dirpath)
    
if not os.path.exists(dirpath+savepath):
    os.makedirs(dirpath+"/"+savepath)

In [10]:
ftp = FTP('data-out.unavco.org')

In [11]:
ftp.login()#unavco's ftp is public, no need for username/pass

'230 Any password will work'

In [12]:
fnames=pd.DataFrame()

In [13]:
fnames=paths[0].str.slice(49)

In [14]:
paths=paths[0].str.slice(26,49)

In [15]:
headerlist= {'col1': ['PR','TD','HR','WS','WD','RI','HI']}
headerdf=pd.DataFrame(data=headerlist)

In [16]:
def file_dance(thefname):
    qf=open(thefname+".csv",'r+')#remove '"', very slow  need to improve  this
    qw=open(thefname+'2.csv','w')
    for line in qf.readlines():
        line=line.replace('"',"")
        qw.write(line)
    os.rename(thefname+".csv",thefname+"t.csv")#rename files/remove dummy file
    os.remove(thefname+"t.csv")
    os.rename(thefname+"2.csv",thefname+".csv")
    qf.close()
    qw.close()
    return

In [17]:
def filemanip(filename,num,totsize):
    gFile = open(filename, "wb")#open file, write binary to my copy
    ftp.retrbinary('RETR '+filename, gFile.write)
    gFile.close()
    os.system("gunzip "+filename)#unzip using system gunzip!!,0 means success
    tmpname=filename[:-2]#remove unix compress extension
    
    p=open(tmpname, "r")#this is to grab the header for later
    s=''
    count=0
    flg=False;
    for line in p.readlines():
        #print(line)
        if '# / TYPES OF OBSERV' in line:#this may be where I process the different headers
            line=line.replace('7','1')
            for y in range(0,7):
                if not headerdf['col1'][y]==headerrem:
                    line=line.replace(headerdf['col1'][y],'  ')
            flg=True
        
        if not flg:
            s+=line;
        elif flg and headerrem in line:
            s+=line
        count=count+1;
        if ' END OF HEADER' in line:
            s+=line
            break
    #print("count="+str(count))#and how many lines the header is
    p.close()
    df=pd.read_fwf(tmpname,header=None,sep=" ",skiprows=count,widths=[3,3,3,3,3,3,7,7,7,7,7,7,7])
    #spacing is fixed so this should work for all files discounting the string header
    df.columns=my_col_lab
    #rename columns for convienence, header is in the origionalrinex file
    
    #append header and dataframe
    dff=pd.DataFrame()
    dff = dff.append({'string_values': s}, ignore_index=True)
    dff=dff.append(df[['year','month','day','hour','minute','second',select]],sort=False)
    dff.reset_index(inplace=True,drop=True)
    
    #spacing and removal of nan's, to get correct string
    dff=dff.fillna(7777)#this value is dummy and should hopefully be unique, to replace
    dff['year']=dff['year'].astype(int)
    dff['month']=dff['month'].astype(int)
    dff['day']=dff['day'].astype(int)
    dff['hour']=dff['hour'].astype(int)
    dff['minute']=dff['minute'].astype(int)
    dff['second']=dff['second'].astype(int)
    dff.replace(to_replace=7777,value='',inplace=True)#replace
    
    #deal with spacing, adds " in string conversion, lambda applied to each cell, *try to remove*
    dff['month']=dff['month'].apply(lambda x: str(x)if x!='' and int(x)>=10 else ' ' +str(x))
    dff['day']=dff['day'].apply(lambda x: str(x) if x!='' and int(x)>=10 else ' ' +str(x))
    dff['hour']=dff['hour'].apply(lambda x: str(x) if x!='' and int(x)>=10 else ' ' +str(x))
    dff['minute']=dff['minute'].apply(lambda x: str(x) if x!='' and int(x)>=10 else ' ' +str(x))
    dff['second']=dff['second'].apply(lambda x: str(x) if x!='' and int(x)>=10 else ' ' +str(x))
    
    #save new file
    fname=filename[:-6]
    dff.to_csv(savepath+'/'+fname+".csv",sep=" ",header=None,index=False)
    
    #remove '"' and rename for proper format
    file_dance(savepath+'/'+fname)
    print(str(totsize-x)+'    ',end='\r')
    return

In [18]:
os.chdir(dirpath)

In [19]:
start_time = time.time()
for x in range(0,paths.size):#manipulate every file from every path
    ftp.cwd(paths[x])#how I navigate
    filemanip(fnames[x],x,paths.size)
    ftp.cwd("/")#reset navigation
print('------------finished------------')
print("tot time: %s seconds" % (time.time() - start_time))

------------finished------------
tot time: 259.0575759410858 seconds


In [20]:
#when this hits 1 you are done for that stations measurement for that year

In [21]:
os.getcwd()#should contain our created directory

'/Users/nickvancise/Desktop/testdir'

In [22]:
#os.chdir("..")#if dir to zip is be 1 lvl away
#os.getcwd()

In [23]:
shutil.make_archive("testunavarch", 'zip', savepath)#zip for download/upload

'/Users/nickvancise/Desktop/testdir/testunavarch.zip'

In [24]:
def diff_test():
    count=0
    not_ident=False
    #str_list=[]
    for name in fnames:
        count=count+1
        tmpname=name[:-2]
        dftest1=pd.read_fwf(tmpname,skiprows=14,sep='+s/',header=None)#orig file
        #print(dftest1[6])
        tmpname=tmpname[:-4]+'.csv'
        dftest2=pd.read_fwf(savepath+'/'+tmpname,skiprows=9,sep='+s/',header=None)#my file
        #print(dftest2[6])
        
        print(str(count)+'           ',end='\n' if (count==fnames.size) else '\r')
        orig_idx=[0,1,2,3,4,5,6] #have to manually set these at the moment from datafile
        my_idx=[0,1,2,3,4,5,6]
        for px in range(len(my_idx)):
            if not dftest1[orig_idx[px]].equals(dftest2[my_idx[px]]):
                print ("The lists are not identical: "+name[:-2]+" "+tmpname+' col:'+str(px))#col is  zero indexed
                not_ident=True
                #return False
    
    if not_ident:
        return False
    return True

In [25]:
#will print the number of files it has checked, and all the files with differences
#to their corresponding pair are listed, and check flag is set

In [26]:
if run_diff_test:
    diff_start_time = time.time()
    b=diff_test()
    if b:
        print('all files are identical')
    print("diff time: %s seconds" % (time.time() - diff_start_time))

365           
all files are identical
diff time: 10.463222026824951 seconds
